# Estatísticas do Jogos da plataforma Alite

  - This file is part of  program Alite.
  - Copyright © 2023  Carlo Oliveira <carlo@nce.ufrj.br>.
  - [Labase](http://labase.selfip.org) - [NCE](http://portal.nce.ufrj.br) - [UFRJ](http://www.ufrj.br).
  - SPDX-License-Identifier: ([GNU General Public License v3.0 or later](https://www.gnu.org/licenses/gpl-3.0-standalone.html)).

## Version and configurations.

 This module retrieves data and plot statistics for the [Alite - Games](https://games.alite.selfip.org).

## [23.07](https://github.com/labase/data_edu/releases)

### Added

- Review oposition deviation and transamb (6).

## [23.06](https://github.com/labase/data_edu/releases)

### Added

- Preliminary stats on Alite games data.
- Classe adicionada para plotar wisconsin (14).



## Recebe dados dos games de alite e converte para dataframe

In [ ]:
import urllib.request, json
import pandas as pd

furl = 'https://games.alite.selfip.org/score/players'
with urllib.request.urlopen(furl) as url:
    data = json.loads(url.read().decode())
    df = pd.DataFrame(data)

df_players = df
print(df)

## Converte lista de dicionários numa lista de nomes dos jogos

In [ ]:
dfg = [[g["game"] for g in t] for t in df.games]
# dfg = pd.DataFrame(dfg)
df['games_l'] = [[g["game"] for g in t] for t in df.games]
df

## Contagem dentra das listas dos jogos

In [ ]:
df['eica'] = [sum([1 for g in t  if "game" in g]) for t in df.games_l.tolist()]
df['wcst'] = [sum([1 for g in t  if "wcst" in g]) for t in df.games_l.tolist()]
df

## Plotagem dos Jogos jogados por Jogador

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
dfx = df.explode('games_l')
# dfm = pd.melt(df, 'games_l')
_ = plt.figure(figsize=(15,8))
chart = sns.countplot(data=dfx, x="name", hue="games_l")
_ = chart.set(title='Contagem dos Jogos', ylabel='Número de Jogos', xlabel="Participantes")
_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')


In [ ]:
df

## Converte ObjectId para Identificador

In [ ]:
import re
matches = [re.findall(r"'(.+?)'",text)[0] for text in df._id.to_list()]
df["oid"] = matches
dfo = df
df.oid[1]

## Testa a recuperação dos dados de um jogador

In [ ]:
import urllib.request, json
import pandas as pd

furl = 'https://games.alite.selfip.org/score/player?oid=6477cf19f626d3cb95e08c92'
with urllib.request.urlopen(furl) as url:
    data = json.loads(url.read().decode())
    df = pd.DataFrame(data)
print(df)

## Limpeza dos dados dos games

In [ ]:
df0 = pd.DataFrame(df.games.to_list())
df0
# df.games

## Obtém o identificador de pontuação em formato texto

In [ ]:
df0['score'] = [re.findall(r"'(.+?)'",text)[0] for text in df0.scorer.to_list()]
df0

## Recupera os dados de jogadas de um jogador

In [ ]:
import urllib.request, json
import pandas as pd

furl = 'https://games.alite.selfip.org/score/games?oid=6477cf19f626d3cb95e08c92'
with urllib.request.urlopen(furl) as url:
    data = json.loads(url.read().decode())
    dfg = pd.DataFrame(data)
print(dfg)

## Expande a lista interna de games e transforma em colunas

In [ ]:
dfx = dfg.explode('scorer')
dfxx = dfx.drop(columns=["scorer"], inplace=False).join(
    dfx.apply(lambda x: pd.Series(x["scorer"].values(), index=x["scorer"].keys()), axis=1))
dfxx

In [ ]:
dfxl = pd.DataFrame(dfx.scorer.values.tolist())
dfx = dfx.drop(columns=["scorer"], inplace=False).reset_index()
dfxj = dfx.join(dfxl)
dfxj

# Recuperação e plotagem das jogadas

## Pedido dos dados ao servidor e converte num arcabouço de dados

In [ ]:
import urllib.request, json
import pandas as pd
furl = 'https://games.alite.selfip.org/score/games?oid={}'
dfa = pd.DataFrame()
dfl = []

def retrieve_games(player):
    global dfl
    with urllib.request.urlopen(furl.format(player)) as urlp:
        dfl.extend(json.loads(urlp.read().decode()))
def process_df():
    dfg_ = pd.DataFrame(dfl)

    dfx_ = dfg_.explode('scorer')
    dfxl_ = [d for d in dfx_.scorer.values.tolist() if isinstance(d, dict)]
    dfxl_ = pd.DataFrame(dfxl_)
    dfx_ = dfx_.drop(columns=["scorer"], inplace=False).reset_index()
    return dfx_.join(dfxl_)
# retrieve_games('6477cf19f626d3cb95e08c92')
games = [retrieve_games(oid) for oid in dfo.oid.to_list()]
# print(dfl)
# dfg = pd.DataFrame(dfl)
# dfx = dfg.explode('scorer')
# dfl = dfx.scorer.values.tolist()
# isinstance(dfl[1712], dict)
# dfl = [d for d in dfl if isinstance(d, dict)]
# pd.DataFrame(dfl)
# dfa = process_df()
# dfa

KeyboardInterrupt: 

## Gera o arcabouço de dados das jogadas

In [ ]:
dfa = process_df()

dfa

## Separa os dados do Wisconsin

In [ ]:
dfwc = dfa.loc[dfa['game'] == 'wcst']
dfwc

## Gráfico de contagem de pontos no Wisc

### Classe para concentrar as operações de arcabouço

In [ ]:
from pandas import DataFrame
from collections import namedtuple
from dataclasses import dataclass

@dataclass
class Plot:
    kind = "value plot factor violin hist heat".split()
    icon = [(6, 5), (6, 5), (3, 2), (6, 0), (2, 2), (5, 4)]
    value = dict(col='valor', title='Contagem dos Valores Wisc', ylabel='Contagem de Valores',
                 xlabel="Participantes", sub_title="Counting Plot de valores do teste Wisconsin")
    plot = dict(col='ponto', title='Contagem dos Pontos Wisc', ylabel='Contagem de Pontos',
                xlabel="Participantes", sub_title="Counting Plot de pontos do teste Wisconsin")
    factor = dict(col='ponto', title='Factor Plot das medidas do Wisc', ylabel='frequência das incidência',
                  xlabel="incidência das medidas", sub_title="Factor Plot do teste Wisconsin")
    violin = dict(col='ponto', title='Violin Plot das medidas do Wisc', ylabel='frequência das incidência',
                  xlabel="incidência das medidas", sub_title="Violin Plot do teste Wisconsin")
    hist = dict(col='ponto', title='Histograma das medidas do Wisc', ylabel='frequência das incidência',
                xlabel="incidência das medidas", sub_title="Histogram Plot do teste Wisconsin")
    heat = dict(col='ponto', title='Correlação das medidas do Wisc', ylabel='listagem das incidência',
                xlabel="listagem das medidas", sub_title="Heatmap Plot dos fatores no teste Wisconsin")
    plotting = dict(value=value, plot=plot, factor=factor, violin=violin, hist=hist, heat=heat)

import numpy as np
class WiscPlot:
    Cfplot = namedtuple("Cfplot", "col title ylabel xlabel")
    Pnt = namedtuple("Pnt", "ok no td")
    Val = namedtuple("Val", "cc cf cn ct")
    def __init__(self, game_url='https://games.alite.selfip.org/score/games?oid={}'):
        self.game_url = game_url
        self.df: DataFrame = DataFrame()
        self.game_data = []
        self.count = 0

    def retrieve_games(self, player):
        """ Retrieve from remote source data for a given player

        :param player: Given player id; identification
        :return: None
        """
        with urllib.request.urlopen(self.game_url.format(player)) as urlp:
            self.game_data.extend(json.loads(urlp.read().decode()))

    def process_df(self):
        """ Shapes current dataframe to filter wisc and replace the column scorer

        :return:
        """
        dfg_ = DataFrame(self.game_data)
        dfg_ = dfg_.loc[dfg_['game'] == 'wcst']
        dfg_["name"] = dfg_.name.apply(lambda x: f"{n[0]} {n[1][:2]}." if len(n := x.split()) > 1 else x)

        dfx_ = dfg_.explode('scorer')
        dfl_ = DataFrame(dfx_.scorer.values.tolist())
        dfx_ = dfx_.drop(columns=["scorer"], inplace=False).reset_index()
        return dfx_.join(dfl_)

    def get_all_games(self, player_oids):
        """ Get all games from a given player.

        :param player_oids: Given player id
        :return: None (assign to current df attribute)
        """
        _ = [self.retrieve_games(oid) for oid in player_oids]
        # print(self.game_data)
        self.df = self.process_df()

    def retrieve_oid_from_person_df(self, person_df):
        """ Trim id vale from extra surroundings.

        :param person_df: Given player dataframe
        :return: self (this object)
        """
        import re
        # noinspection PyProtectedMember
        oid_list = [re.findall(r"'(.+?)'", text)[0] for text in person_df._id.to_list()]
        self.get_all_games(oid_list)
        return self

    def refine_point_value_info(self):
        """ Extract some cognitive relevant properties from existing data,

        :return: transformed dataframe with new columns
        """

        def counter(a, b):
            """ Counts repetition of a figure.

            :param a: current figure of the series
            :param b: next figure of the series
            :return: count if prevails or zero if not
            """
            a, b = int(a), int(b)
            self.count += (1 if a else 0)
            count, self.count = self.count if not b else 0, 0 if not b else self.count
            return count
        def alter(a, b):
            a, b = int(a), int(b)
            no_alter = (a == b) or (b==0) or (a==0)
            self.count += (1 if a!=b else 0)
            count, self.count = self.count if no_alter else 0, 0 if no_alter else self.count
            return count

        def joiner(k, t, w):
            k, t = int(k), int(t)
            return int(int(t != w) * 2 ** w * k)

        def nobother(cc, cf, cn, ct):
            non =  [int(cc), int(cf), int(cn)]
            do = non.pop(int(ct)%3) < 1
            return int(all(non)) if do else 0

        def noner(cc, cf, cn, ct):
            non =  [int(cc), int(cf), int(cn)]
            do = non.pop(int(ct)%3) < 1
            return (non[0] + 2* non[1]) if do else 0
            # return int(any(non)) if do else 0
            # return 1 if do else 0

        def bother(c, f, n, t, e=True):
            c, f, n, t = int(c), int(f), int(n), int(t) % 3
            all_k = [c, f, n]
            if t > 2 or t < 0:
                raise ValueError(t)
            target = all_k.pop(t)
            return target * sum(all_k) if e else int(all(all_k))

        point_list = [self.Pnt(text[:-2], *list(text[-2:])) for text in self.df.ponto.to_list()]
        new_list = point_list[1:] + [self.Pnt(0, 0, 0)]
        val_list0 = [self.Val(*list(text)) for text in self.df.valor.to_list()]
        non_list = [noner(*list(val)) for val in self.df.valor.to_list()]
        nob_list = [nobother(*list(val)) for val in self.df.valor.to_list()]
        val_list = [joiner(val.cc, val.ct, 0) + joiner(val.cf, val.ct, 1) + joiner(val.cn, val.ct, 2)
                    for val in
                    val_list0]
        conservation = [counter(a.ok, b.ok) for a, b in zip(point_list, new_list)]
        perseveration = [counter(a.no, b.no) for a, b in zip(point_list, new_list)]
        deviation = [alter(a, b) for a, b in zip(non_list, non_list[1:] + [0])]
        oposition = [counter(a, b) for a, b in zip(nob_list, nob_list[1:] + [0])]
        ambiguation = [bother(c, f, n, t) for c, f, n, t in val_list0]
        transamb = [bother(c, f, n, t, False) for c, f, n, t in val_list0]
        zipped = list(zip(conservation, perseveration, oposition, deviation, ambiguation, transamb))
        df__ = DataFrame(zipped, columns='conservation perseveration oposition deviation ambiguation transamb'.split())
        _df = self.df.drop(columns='game goal trial carta casa move time ponto valor'.split(),
                           inplace=False).reset_index()
        _df = _df.join(df__).drop(columns='level_0 index _id'.split(), inplace=False)
        # self.df = _df
        return _df

    def plot_template(self, cfg: Cfplot, runner, x_lim=None, y_lim=None, tick=False, tl= False, df=None):
        """ Template method to embrace a given method.

        :param tick: Add slant labels to x-axis.
        :param cfg: Plotting configuration
        :param runner: Given method to be templated.
        :param df: Dataframe source for plotting.
        :param x_lim: Limits for x-axis.
        :param y_lim: Limits for y-axis.
        :return: Plotting context.
        """
        from matplotlib import pyplot as plt_
        f = plt_.figure(figsize=(20, 8))
        ax = f.add_subplot(1, 1, 1)

        if df is None:
            df_ = self.refine_point_value_info()
            df_ = pd.melt(df_, id_vars="name", var_name="measure", value_name="incidence")
        else:
            df_ = df
        chart_ = runner(df_, ax)
        _ = chart_.set(title=cfg.title, ylabel=cfg.ylabel, xlabel=cfg.xlabel)
        # _ = chart_.set_xticklabels(chart_.get_xticklabels(), rotation=45, horizontalalignment='right')
        chart_.set_xticklabels(chart_.get_xticklabels(), rotation=45, horizontalalignment='right') if tl else None
        _ = chart_.set_xticklabels(rotation=45, horizontalalignment='right') if tick else None
        chart_.set(xlim=x_lim) if x_lim else None
        chart_.set(ylim=y_lim) if y_lim else None
        # chart_.set_xlim(left=x_lim[0], right=x_lim[1]) if x_lim else None
        return plt_

    def plot(self, cfg: Cfplot):
        """ Counting bar plot

        :param cfg: Plotting configuration
        :return: Plotting context.
        """
        import seaborn as sbn
        return self.plot_template(cfg, lambda df_, a: sbn.countplot(data=df_, x="name", hue=cfg.col), df=self.df,
                                  tl=True)

    def factorplot(self, cfg: Cfplot):
        """ Factor bar plot

        :param cfg: Plotting configuration
        :return: Plotting context.
        """
        import seaborn as sbn
        return self.plot_template(cfg, lambda df_, a: sbn.catplot(
            x='name', y='incidence', hue='measure', data=df_, kind='bar', height=8, aspect=2.5),
                                  tick=True, y_lim=(0, 1))

    def violinplot(self, cfg: Cfplot):
        """ Violin gaussian plot

        :param cfg: Plotting configuration
        :return: Plotting context.
        """
        import seaborn as sbn
        return self.plot_template(cfg, lambda df_, a: sbn.boxplot(
            x='name', y='incidence', hue='measure', data=df_), y_lim=(-1, 2), tl=True)

    def histplot(self, cfg: Cfplot):
        """ Histogram bar plot

        :param cfg: Plotting configuration
        :return: Plotting context.
        """
        import seaborn as sbn
        return self.plot_template(cfg, lambda df_, a: sbn.histplot(
            data=df_, stat="count", multiple="stack",
            x="incidence", kde=False,
            palette="pastel", hue="measure",
            element="bars", legend=True, ax=a),
                                  x_lim=(9, None), y_lim=(0, 8))

    def heatmap(self, cfg: Cfplot):
        """ Correlation map for cognition profiles

        :param cfg: Plotting configuration
        :return: Plotting context.
        """
        from matplotlib import pyplot as plt_
        import numpy as np
        plt_.figure(figsize=(15, 8))
        df_ = self.refine_point_value_info()
        df_ = df_.drop(columns=['name'], inplace=False)
        # Compute the correlation matrix
        corr = df_.corr()
        # Generate a mask for the upper triangle
        mask = np.triu(np.ones_like(corr, dtype=bool))
        # Generate a custom diverging colormap
        cmap = sns.diverging_palette(230, 20, as_cmap=True)
        # Draw the heatmap with the mask and correct aspect ratio
        # chart_ = sns.heatmap(corr, cmap=cmap, vmax=.3, center=0,
        chart_ = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                             square=True, linewidths=.5, cbar_kws={"shrink": .5})
        _ = chart_.set(title=cfg.title, ylabel=cfg.ylabel, xlabel=cfg.xlabel)
        return plt_

    def run_plotting(self, kind, data_frame):
        """ Plotting service selector

        :param kind: Selection key
        :param data_frame: Data to be plotted.
        :return: Plotting context
        """
        plotters = dict(value=self.plot, plot=self.plot, factor=self.factorplot,
                        violin=self.violinplot, hist=self.histplot, heat=self.heatmap)
        # from dash import Plotting as Plot
        configurations = dict(Plot.plotting)
        configurations[kind].pop("sub_title") if "sub_title" in configurations[kind] else None
        self.retrieve_oid_from_person_df(data_frame)
        return plotters[kind](WiscPlot.Cfplot(**configurations[kind]))

# out = WiscPlot().rerieve_oid_from_person_df(df_players).refine_point_value_info()
conf0 = WiscPlot.Cfplot(
     col='ponto', title='Contagem dos Pontos Wisc', ylabel='Contagem de Pontos', xlabel="Participantes")
# out = WiscPlot().rerieve_oid_from_person_df(df_players).factorplot(conf)
conf = WiscPlot.Cfplot(
     col='ponto', title='Histograma das medidas do Wisc', ylabel='frequência das incidência', xlabel="incidência das medidas")
out = WiscPlot().retrieve_oid_from_person_df(df_players).histplot(conf)
# out = WiscPlot().rerieve_oid_from_person_df(df_players).violinplot(conf)
# out = WiscPlot().rerieve_oid_from_person_df(df_players).heatmap(conf)
# out = WiscPlot().rerieve_oid_from_person_df(df_players).plot(conf)
# print(out)
# WiscPlot().rerieve_oid_from_person_df(df_players).refine_point_value_info()

## KDE Plot

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
sns.kdeplot(
    data=WiscPlot().retrieve_oid_from_person_df(df_players).refine_point_value_info(),
    x="conservation",
    y="deviation",
    hue="name",
    thresh=.1,
)
ax.set_xlim((-1,2.5))  #
ax.set_ylim((-1,2.5))

## Scatter plot

In [ ]:
sns.relplot(hue="name", x="conservation", y="deviation", size="oposition",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=10, data=WiscPlot().retrieve_oid_from_person_df(df_players).refine_point_value_info())


## Joint Plot

In [ ]:
g = sns.jointplot(
    data= WiscPlot().retrieve_oid_from_person_df(df_players).refine_point_value_info()
,
    x="conservation", y="ambiguation", hue="name", height=10, xlim = (-2,3),
    kind="kde",
)
# g.set(xlim=(-8,8))

## Joint oposition deviation

In [ ]:
g = sns.jointplot(
    data= WiscPlot().retrieve_oid_from_person_df(df_players).refine_point_value_info()
,
    x="oposition", y="deviation", hue="name", height=10, ylim = (-1,3.5), xlim = (-1,1.5),
    kind="kde",
)


## Pair Plot

In [ ]:
sns.set_theme(style="ticks")
dfp = WiscPlot().retrieve_oid_from_person_df(df_players).refine_point_value_info()
# df = sns.load_dataset("penguins")
sns.pairplot(dfp, hue="name")


## Factor plot

In [ ]:
out = WiscPlot().retrieve_oid_from_person_df(df_players).factorplot(conf)


### Plotagem das contagens dos valores do Wiconsin

In [ ]:
conf = WiscPlot.Cfplot(
     col='valor', title='Contagem dos Valores Wisc', ylabel='Contagem de Valores', xlabel="Participantes")
out = WiscPlot().retrieve_oid_from_person_df(df_players).plot(conf)


### Plotagem das contagens dos pontos do Wiconsin

In [ ]:
conf = WiscPlot.Cfplot(
     col='ponto', title='Contagem dos Pontos Wisc', ylabel='Contagem de Pontos', xlabel="Participantes")
out = WiscPlot().retrieve_oid_from_person_df(df_players).plot(conf)


### Gráfico dos fatores das medidas do Wiconsin

In [ ]:
conf = WiscPlot.Cfplot(
     col='ponto', title='Contagem dos Pontos Wisc', ylabel='Contagem de Pontos', xlabel="Participantes")
out = WiscPlot().retrieve_oid_from_person_df(df_players).factorplot(conf)


### Gráfico de violino das medidas do Wiconsin

In [ ]:
conf = WiscPlot.Cfplot(
    col='ponto', title='Gráfico de Violino das medidas do Wisc', ylabel='frequência das incidência',
    xlabel="incidência das medidas")
out = WiscPlot().retrieve_oid_from_person_df(df_players).violinplot(conf)


### Histograma das medidas do Wiconsin

In [ ]:
conf = WiscPlot.Cfplot(
     col='ponto', title='Histograma das medidas do Wisc', ylabel='frequência das incidência', xlabel="incidência das medidas")
out = WiscPlot().retrieve_oid_from_person_df(df_players).histplot(conf)


### Correlação das medidas do Wiconsin

In [ ]:
conf = WiscPlot.Cfplot(
     col='ponto', title='Correlação das medidas do Wisc', ylabel='lista das incidência', xlabel="lista das incidências")
out = WiscPlot().retrieve_oid_from_person_df(df_players).heatmap(conf)


In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
_ = plt.figure(figsize=(15,8))
chart = sns.countplot(data=dfwc, x="name", hue="ponto")
_ = chart.set(title='Contagem dos Pontos Wisc', ylabel='Contagem de Pontos', xlabel="Participantes")
_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

## Gráfico de distribuição dos pontos no Wisc

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
pd.to_numeric(dfwc.ponto)
_ = plt.figure(figsize=(15,8))
chart = sns.displot(data=dfwc, x="ponto")
_ = chart.set(title='Distribuição dos Pontos Wisc', ylabel='Contagem de Pontos', xlabel="Pontuações")

## Gráfico da distribuição dos valores no Wisc

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
pd.to_numeric(dfwc.ponto)
_ = plt.figure(figsize=(15,8))
chart = sns.displot(data=dfwc, x="valor")
_ = chart.set(title='Distribuição dos Valores Wisc', ylabel='Contagem de Valores', xlabel="Valores")

## Gráfico da correlação entre pontos e valores no Wisc

In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
def jitter(values,j):
    return values + np.random.normal(j,10,values.shape)
cols = ['valor', 'ponto', 'carta']
dfwc[cols] = dfwc[cols].apply(pd.to_numeric, errors='coerce', axis=1)
# dfwc["ponto"] = pd.to_numeric(dfwc.ponto)
# pd.to_numeric(dfwc.valor)
_ = plt.figure(figsize=(15,8))
chart = sns.scatterplot(data=dfwc, x=jitter(dfwc.valor, 100), y=jitter(dfwc.ponto, 50), size="carta", hue="trial")
_ = chart.set(title='Correlação dos Valores Wisc', ylabel='Valores', xlabel="Pontos")

## Gráfico da contagem de valores do Wisc

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
_ = plt.figure(figsize=(15,8))
chart = sns.countplot(data=dfwc, x="name", hue="valor")
_ = chart.set(title='Contagem dos Valores Wisc', ylabel='Contagens de Valor', xlabel="Participantes")
_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

## Limpeza dos nomes dos valores de carta e ponto no EICA

In [ ]:
dfei = dfa.loc[dfa['game'] == 'game']
dfei['carta'] = [re.findall(r"__(.+?)__",text)[0] if type(text) == str and text.startswith('_') else text for text in dfei.carta.to_list()]
dfei['ponto'] = [re.findall(r"_(.+?)_",text)[0] if type(text) == str and text.startswith('_') else text for text in dfei.ponto.to_list()]

dfei

## Gráfico de contagem de pontos EICA

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
_ = plt.figure(figsize=(15,8))
chart = sns.countplot(data=dfei, x="name", hue="ponto")
_ = chart.set(title='Contagem de Pontos do Eica', ylabel='Contagens de Pontos', xlabel="Participantes")
_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

## Gráfico de contagem de cartas do EICA

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
_ = plt.figure(figsize=(15,8))
chart = sns.countplot(data=dfei, x="name", hue="carta")
_ = chart.set(title='Contagem de Cartas do Eica', ylabel='Contagens de Carta', xlabel="Participantes")
_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')